# AEMO and WU (weather data 30 mins interval) Data Merging

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import os

In [2]:
#loading our electricity demand data
df = pd.read_csv("../Data/edited/AEMO_merged/combined_csv-operational-demand.csv")
df = df.sort_values('timestamp')
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.head()

,timestamp,Operational Demand (MW)
0,2006-09-20 00:00:00,1347.418
1,2006-09-20 00:30:00,1312.708
2,2006-09-20 01:00:00,1298.644
3,2006-09-20 01:30:00,1288.972
4,2006-09-20 02:00:00,1282.006


In [3]:
#loading our weather data from WU
df_weather = pd.read_csv("../Data/edited/WU_merged/wu_weather_merged.csv")
df_weather = df_weather.loc[:, ['Datetime', 'Temperature', 'Dew Point', 'Humidity', 'Wind', 'Wind Speed', 'Wind Gust', 'Pressure', 'Precip.', 'Condition']]
df_weather.head() 

,Datetime,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition
0,2009-05-22 00:00:00,15.00,7.78,63,W,15,0,29,0,Partly Cloudy
1,2009-05-22 00:30:00,15.00,7.78,63,W,18,0,29,0,Mostly Cloudy
2,2009-05-22 00:54:00,15.00,6.11,55,WNW,18,33,29,0,Mostly Cloudy
3,2009-05-22 01:00:00,15.00,7.22,59,W,21,33,29,0,Mostly Cloudy / Windy
4,2009-05-22 01:30:00,13.89,7.22,63,W,18,31,29,0,Mostly Cloudy


In [4]:
df_weather = df_weather.rename(columns= {'Datetime' : 'timestamp'})
df_weather['timestamp'] = pd.to_datetime(df_weather['timestamp'])

In [5]:
#as our weather data is not consistent, we are group in 30 mins interval consitent with AEMO data
df_weather =df_weather.groupby(pd.Grouper(key='timestamp', freq='30min')).mean().dropna()
df_weather.head()

,Temperature,Dew Point,Humidity,Wind Speed,Wind Gust,Pressure,Precip.
timestamp,,,,,,,
2006-09-20 00:00:00,13.89,12.22,88.0,5.0,0.0,30.0,0.0
2006-09-20 00:30:00,13.89,12.22,88.0,13.0,0.0,30.0,0.0
2006-09-20 01:00:00,13.89,10.00,77.0,8.0,0.0,30.0,0.0
2006-09-20 01:30:00,13.89,11.11,82.0,8.0,0.0,30.0,0.0
2006-09-20 02:00:00,15.00,12.22,82.0,10.0,0.0,30.0,0.0


In [6]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.head()

,timestamp,Operational Demand (MW)
0,2006-09-20 00:00:00,1347.418
1,2006-09-20 00:30:00,1312.708
2,2006-09-20 01:00:00,1298.644
3,2006-09-20 01:30:00,1288.972
4,2006-09-20 02:00:00,1282.006


In [7]:
df.isna().sum()

timestamp                  0
Operational Demand (MW)    0
dtype: int64

In [8]:
#merging 2 dataset with merge_asof to make sure that if the interval not matching, it will at the closest interval
df_merged = pd.merge_asof(df_weather.sort_values('timestamp'),df.sort_values('timestamp'), on='timestamp',allow_exact_matches=True)

In [9]:
#a preview of our merged dataset
df_merged.head()

,timestamp,Temperature,Dew Point,Humidity,Wind Speed,Wind Gust,Pressure,Precip.,Operational Demand (MW)
0,2006-09-20 00:00:00,13.89,12.22,88.0,5.0,0.0,30.0,0.0,1347.418
1,2006-09-20 00:30:00,13.89,12.22,88.0,13.0,0.0,30.0,0.0,1312.708
2,2006-09-20 01:00:00,13.89,10.00,77.0,8.0,0.0,30.0,0.0,1298.644
3,2006-09-20 01:30:00,13.89,11.11,82.0,8.0,0.0,30.0,0.0,1288.972
4,2006-09-20 02:00:00,15.00,12.22,82.0,10.0,0.0,30.0,0.0,1282.006


In [10]:
#Checking any na values in the newly merged dataset
df_merged.isna().sum()

timestamp                  0
Temperature                0
Dew Point                  0
Humidity                   0
Wind Speed                 0
Wind Gust                  0
Pressure                   0
Precip.                    0
Operational Demand (MW)    0
dtype: int64

In [11]:
#save the csv to folder
df_merged.to_csv("../Data/edited/aemo_wu_merged.csv")